In [11]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
def load_images_from_folder(folder, img_size) :
    images = []
    labels = []
    label_names = os.listdir(folder)
    for label, class_name in enumerate(label_names) :
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder) :
            img_path = os.path.join(class_folder, filename)
            img = Image.open(img_path).convert("RGB").resize(img_size)
            images.append(np.array(img) / 255.0)
            labels.append(label)
    return np.array(images), np.array(labels), label_names

def create_cnn_model(input_shape, num_classes) :
    model = Sequential()
    model.add(Conv2D(32, kernel_size = (3, 3), activation = "relu", input_shape = input_shape))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size = (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size = (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation = "softmax"))
    model.compile(optimizer = "adam",
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])
    return model

In [12]:
data_folder = ""
images, labels, label_names = load_images_from_folder(data_folder)

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes = len(label_names))
y_test = to_categorical(y_test, num_classes = len(label_names))

input_shape = x_train.shape[1:]
num_classes = len(label_names)

cnn_model = create_cnn_model(input_shape, num_classes)

cnn_model.fit(x_train, y_train, epochs = 20, batch_size = 32, validation_split = 0.2)

loss, accuracy = cnn_model.evaluate(x_test, y_test)
print(f"Test Accuracy : {accuracy:.2f}")

TypeError: load_images_from_folder() missing 1 required positional argument: 'img_size'

In [13]:
def predict_image(model, image_path, label_names, img_size = (128, 128)) :
    img = Image.open(image_path).convert("RGB").resize(img_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis = 0)
    predictions = model.predict(img_array)
    predicted_label = label_names[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_label, confidence

In [14]:
test_image_path = ""
predicted_label, confidence = predict_image(cnn_model, test_image_path, label_names)
print(f"Predicted Label : {predicted_label}, Confidence : {confidence:.2f}")

NameError: name 'cnn_model' is not defined